<a href="https://colab.research.google.com/github/Bajanar3/Grammar-Nazi/blob/master/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function

In [20]:
try:
  %tensorflow_version 2.x
except: Exception
pass


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import tensorflow as tf
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd

keras = tf.keras
from keras import Sequential as model
from keras.layers import Dense, Conv1D, InputLayer, MaxPooling1D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam


In [0]:
def get_dataset_files (filepath):
  data = pd.read_excel(filepath, na_values=9999)
  dataset = data.to_numpy()
  return dataset

def CNN_Model(training_set, validation_set, filter, kernel_size, epoch,  stride=1, padding='casual', activation='relu', patience=5):
  model.add(InputLayer(input_shape=[None,1]))
  model.add(Conv1D(filters=filter, 
                   kernel_size=kernel_size, 
                   strides=stride,
                   padding=padding, 
                   activation=activation
                   ))
  model.add(MaxPooling1D(pool_size=2, 
                         padding='valid'))
  model.add(Conv1D(filters=16, 
                   kernel_size=5,
                   strides=1,
                   padding=padding,
                   activation=activation
                   ))
  model.add(MaxPooling1D(pool_size=2, 
                         padding='valid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='Adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=patience)
  model_checkpoint = keras.callbacks.ModelCheckpoint('MyCNN_checkpoint', save_best_only=True )
  history = model.fit(x=training_set, y=validation_set, batch_size=None, epochs=epoch, validation_split=0.2, callbacks=[callback, model_checkpoint])
  return history 


def CNNModel_forecast(model_filepath, series, window_size):
    model = keras.models.load_model(filepath=model_filepath)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast
